# Joining Tables

In [1]:
import warnings
warnings.filterwarnings('ignore')

from reframe import Relation
r1 = Relation('/home/faculty/yasiro01/pub/R1.csv',sep=',')
r2 = Relation('/home/faculty/yasiro01/pub/R2.csv',sep=',')


In [2]:
r1

,C1,C2
0,1,A
1,2,B
2,3,C
3,5,D


In [3]:
r2

,C1,C3
0,7,E
1,3,F
2,1,J
3,2,L


## Cartesian Product

So far we have restricted ourselves to operators that operate on one table (relation) at a time. This is logical in the sense that our operators create relations! However, we know that a typical database contains many tables, which in fact may be related. So, how do we do queries using mulitple tables?

The first step toward applying the operators we have learned so far to multiple tables is to merge the tables together. We do this using the cartesian product.

A cartesian product creates one table out of two tables by creating every possible combination of each row in table A with each row in table B, forming a new relation with A+B columns, and A\*B rows!


In [7]:
r1.cartesian_product(r2)

,C1_x,C2,C1_y,C3
0,1,A,7,E
1,1,A,3,F
2,1,A,1,J
3,1,A,2,L
4,2,B,7,E
5,2,B,3,F
6,2,B,1,J
7,2,B,2,L
8,3,C,7,E
9,3,C,3,F


### Relation 1

COL_1 | COL_2
------|------
 A | B
 C | D
 E | F
 

### Relation 2

COL_3 | COL_4
------|-------
 1 | A
 2 | C
 3 | A
 

### Cartesian Product: Result

COL_1 | COL_2 | COL_3 | COL_4
------|-------|-------|------
 A | B | 1 | A
 A | B | 2 | C
 A | B | 3 | A
 C | D | 1 | A
 C | D | 2 | C
 C | D | 3 | A
 E | F | 1 | A
 E | F | 2 | C
 E | F | 3 | A
 

### Cartesian Product: Challenge

Of course this can create an **enormous** table, so the cartesian product is always followed by a query where we limit the number of rows by comparing a column in relation 1 against a column in relation 2.


### Cartesian Product: Query

COL_1 == COL_4

COL_1 | COL_2 | COL_3 | COL_4
------|-------|-------|------
 A | B | 1 | A
 A | B | 3 | A
 C | D | 2 | C


## Natural Join

The natural join or `njoin` operator takes the pattern of cartesian product followed by query, and wraps it all into one operation subject to the following:

* The query condition tests for equality
* The query condition of equality applies to all columns with the same name in both relations

You can see this in the following diagram, where we have two relations. Both have a column named C1.

The resulting relation has a single C1 column where only the rows where C1 holds the same value in both relations. The other values from the row are filled in with the values from the matching rows.


In [9]:
r1.njoin(r2)

,C1,C2,C3
0,1,A,J
1,2,B,L
2,3,C,F


## SQL


In [10]:
%load_ext sql


In [11]:
%%sql

postgresql://yasiro01@localhost/jtest


'Connected: yasiro01@jtest'

### Cartesian Product: SQL


In [12]:
%%sql

select *
from r1, r2;


16 rows affected.


C1,C2,C1_1,C3
1,A,7,E
1,A,3,F
1,A,1,J
1,A,2,L
2,B,7,E
2,B,3,F
2,B,1,J
2,B,2,L
3,C,7,E
3,C,3,F


### Natural Join: SQL

In [13]:
%%sql

select *
from r1 natural join r2;


3 rows affected.


C1,C2,C3
1,A,J
2,B,L
3,C,F


### Natural Join vs Cartesian Product

To return to the cartesian product example, note that it is not 100% equivalent as the cartesian product retains and renames the second copy of column1.


In [20]:
%%sql

select *
from r1, r2
where r1."C1" = r2."C1"

3 rows affected.


C1,C2,C1_1,C3
1,A,1,J
2,B,2,L
3,C,3,F


## Natural Join Example

Now lets look at a more real example. From our city and country tables we have a problem:

* the column name is in both relations, but means different things
* the column population is in both relations but means different things
* the column we would like to join on is the **countrycode** column, but it is called code in the country relation and countrycode in the city relation

We can remedy this in relational algebra by using the rename operator.


In [21]:
city = Relation('/home/faculty/yasiro01/pub/city.csv')
country = Relation('/home/faculty/yasiro01/pub/country.csv')

In [40]:
city.head()

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000.0
1,2,Qandahar,AFG,Qandahar,237500.0
2,3,Herat,AFG,Herat,186800.0
3,4,Mazar-e-Sharif,AFG,Balkh,127800.0
4,5,Amsterdam,NLD,Noord-Holland,731200.0


In [23]:
country.head()

,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,15864000,78.3,371362.0,360478.0,Nederland,Constitutional Monarchy,Beatrix,5.0,NL
2,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.0,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33.0,AN
3,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
4,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,31471000,69.7,49982.0,46966.0,Al-Jazair/Algérie,Republic,Abdelaziz Bouteflika,35.0,DZ


In [28]:
city.rename('name', 'city_name').rename('population', 'city_pop').head()

,id,city_name,countrycode,district,city_pop
0,1,Kabul,AFG,Kabol,1780000.0
1,2,Qandahar,AFG,Qandahar,237500.0
2,3,Herat,AFG,Herat,186800.0
3,4,Mazar-e-Sharif,AFG,Balkh,127800.0
4,5,Amsterdam,NLD,Noord-Holland,731200.0


### Find all cities in Norway

Relational Algebra


In [32]:
country.query('name == "Norway"').project(['code'])

,code
148,NOR


In [34]:
city.query('countrycode == "NOR"')

,id,name,countrycode,district,population
2806,2807,Oslo,NOR,Oslo,508726.0
2807,2808,Bergen,NOR,Hordaland,230948.0
2808,2809,Trondheim,NOR,Sør-Trøndelag,150166.0
2809,2810,Stavanger,NOR,Rogaland,108848.0
2810,2811,Bærum,NOR,Akershus,101340.0


In [43]:
city.rename('name', 'city_name').rename('population', 'city_pop').\
njoin(country.rename('code', 'countrycode')).\
query('name == "Norway"')


,id,city_name,countrycode,district,city_pop,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
2806,2807,Oslo,NOR,Oslo,508726.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2807,2808,Bergen,NOR,Hordaland,230948.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2808,2809,Trondheim,NOR,Sør-Trøndelag,150166.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2809,2810,Stavanger,NOR,Rogaland,108848.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2810,2811,Bærum,NOR,Akershus,101340.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO


In [38]:
city.rename('name', 'city_name').rename('population', 'city_pop').\
njoin(country.rename('code', 'countrycode').query('name == "Norway"'))

,id,city_name,countrycode,district,city_pop,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
0,2807,Oslo,NOR,Oslo,508726.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
1,2808,Bergen,NOR,Hordaland,230948.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
2,2809,Trondheim,NOR,Sør-Trøndelag,150166.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
3,2810,Stavanger,NOR,Rogaland,108848.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO
4,2811,Bærum,NOR,Akershus,101340.0,Norway,Europe,Nordic Countries,323877.0,1905.0,4478500,78.7,145895.0,153370.0,Norge,Constitutional Monarchy,Harald V,2807.0,NO


### Find all cities in Norway

Structured Query Language


In [ ]:
%sql postgresql://yasiro01:@localhost/world


In [ ]:
%%sql



In [ ]:
%%sql



## Movie Database

The natural join operator works very well on the movie database as it has two columns with the same name in both the moviecast table and the release_date table.


### Find the names of all of the lead actors in the  movies released in October of 2015 in Norway

In [ ]:
moviecast = Relation('/home/faculty/yasiro01/pub/cast.csv',sep=',')
release_date = Relation('/home/faculty/yasiro01/pub/release_dates.csv',sep=',')

In [ ]:
moviecast.head()

In [ ]:
release_date.head()

When you run the above query it takes a while, because it is doing a very large join. However we can make it run more quickly by reducing the size of the relations involved in the join by using a query on each.

In [ ]:
%sql postgresql://yasiro01:@localhost/movies


In [ ]:
%%sql

